# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using AtomsBuilder
using DFTK
using LinearAlgebra
using PseudoPotentialData

pseudopotentials = PseudoFamily("dojo.nc.sr.pbesol.v0_4_1.standard.upf")
model = model_DFT(bulk(:Si); functionals=PBEsol(), pseudopotentials)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire in the density
tol = 1e-6

1.0e-6

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.397862487303                   -0.89    5.0   26.0ms
  2   -8.400259061147       -2.62       -1.73    1.0   19.1ms
  3   -8.400401317403       -3.85       -2.90    1.2   19.2ms
  4   -8.400427724762       -4.58       -2.90    3.2   54.7ms
  5   -8.400427950638       -6.65       -3.04    1.0   18.9ms
  6   -8.400428150302       -6.70       -5.02    1.0   18.8ms
  7   -8.400428155880       -8.25       -4.43    3.5   25.7ms
  8   -8.400428156260       -9.42       -5.44    1.2   19.8ms
  9   -8.400428156276      -10.81       -6.22    1.5   20.2ms


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag   Δtime
---   ---------------   ---------   ---------   ----   ----   ------
  1   -8.397784329128                   -0.90           4.5    964ms
  2   -8.400378564449       -2.59       -1.77   0.80    2.2    506ms
  3   -8.400422927669       -4.35       -2.94   0.80    1.0    187ms
  4   -8.400428107744       -5.29       -3.43   0.80    2.2   19.5ms
  5   -8.400428152775       -7.35       -4.83   0.80    1.2   17.1ms
  6   -8.400428156267       -8.46       -5.36   0.80    3.2   21.7ms
  7   -8.400428156276      -11.07       -6.75   0.80    1.0   16.5ms


## Direct minimization

In [4]:
scfres_dm = direct_minimization(basis; tol);

┌ Warning: x_tol is deprecated. Use x_abstol or x_reltol instead. The provided value (-1) will be used as x_abstol.
└ @ Optim ~/.julia/packages/Optim/7krni/src/types.jl:110
┌ Warning: f_tol is deprecated. Use f_abstol or f_reltol instead. The provided value (-1) will be used as f_reltol.
└ @ Optim ~/.julia/packages/Optim/7krni/src/types.jl:120
n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   +0.787045022311                   -1.05    2.42s
  2   -1.899677898645        0.43       -0.61    118ms
  3   -4.689837972909        0.45       -0.40   44.0ms
  4   -6.205767352642        0.18       -0.50   87.2ms
  5   -7.589315887883        0.14       -0.74   43.9ms
  6   -7.907351073719       -0.50       -1.44   32.6ms
  7   -8.211250895508       -0.52       -1.71   32.4ms
  8   -8.291074794018       -1.10       -2.11   56.2ms
  9   -8.346642568920       -1.26       -2.23   32.7ms
 10   -8.367161776604       -1.69       -2.33   32

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=0.5);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.397887576760                   -0.90    5.2   26.1ms


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   -8.400427980102                   -1.78    23.7s
  2   -8.400428156277       -6.75       -4.02    2.19s
  3   -8.400428156277      -14.75       -7.81    109ms


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 6.279734902357257e-7
|ρ_newton - ρ_scfv| = 3.2576285338069753e-7
|ρ_newton - ρ_dm|   = 1.255792358320856e-6
